# Feature Engineering

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns


!pip install plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Defaulting to user installation because normal site-packages is not writeable


In [4]:
%run "variables.py"

In [3]:
%run "functions.ipynb"

# Data loading

Load the two CSV files, pred_maint_timeseries_with_failures.csv into pandas DataFrame.

In [5]:
# df = pd.read_csv(f'{DATA_DIR}/pred_maint_timeseries_with_failures.csv')
df = pd.read_csv(f'{DATA_DIR}/pred_maint_yearly_timeseries.csv')
display(df.head())

,machineid,datetimestamp,enginegastemperature1,enginegastemperature2,enginegastemperature3,enginegastemperature4,enginegastemperature5,enginegastemperature6,pressure
0,1,2025-01-01 00:00:00,302.483571,385.408617,303.238443,307.615149,298.829233,298.829315,103.158426
1,1,2025-01-01 00:10:00,303.837174,300.111109,297.861035,297.340913,299.412622,301.110395,98.464047
2,1,2025-01-01 00:20:00,299.877724,301.777757,302.085056,304.162309,298.533004,299.850807,100.190252
3,1,2025-01-01 00:30:00,303.323272,303.895963,294.494511,305.651141,301.865595,298.067635,97.682460
4,1,2025-01-01 00:40:00,297.766293,307.621208,301.614999,293.032915,289.108329,294.780518,100.345387
